In [1]:
import requests

In [2]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"


In [3]:
data = requests.get(standings_url)
data.text

'<!DOCTYPE html><html lang="en-US"><head><title>Just a moment...</title><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><meta http-equiv="X-UA-Compatible" content="IE=Edge"><meta name="robots" content="noindex,nofollow"><meta name="viewport" content="width=device-width,initial-scale=1"><style>*{box-sizing:border-box;margin:0;padding:0}html{line-height:1.15;-webkit-text-size-adjust:100%;color:#313131;font-family:system-ui,-apple-system,BlinkMacSystemFont,"Segoe UI",Roboto,"Helvetica Neue",Arial,"Noto Sans",sans-serif,"Apple Color Emoji","Segoe UI Emoji","Segoe UI Symbol","Noto Color Emoji"}body{display:flex;flex-direction:column;height:100vh;min-height:100vh}.main-content{margin:8rem auto;padding-left:1.5rem;max-width:60rem}@media (width <= 720px){.main-content{margin-top:4rem}}.h2{line-height:2.25rem;font-size:1.5rem;font-weight:500}@media (width <= 720px){.h2{line-height:1.5rem;font-size:1.25rem}}#challenge-error-text{background-image:url("");background-repeat:no-re

In [4]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(data.text)

In [9]:
from bs4 import Comment
import pandas as pd
comments = soup.find_all(string=lambda text: isinstance(text, Comment))

tables = []
for c in comments:
    if "table" in c:  # look for commented-out tables
        table_soup = BeautifulSoup(c, "html.parser")
        table = table_soup.find("table")
        if table:
            tables.append(table)

# Instead of indexing blindly, target by ID
standings_table = None
for t in tables:
    if t.get("id") == "results2025-202691_overall":  # Premier League standard stats
        standings_table = t
        break

if standings_table:
    df = pd.read_html(str(stats_table))[0]
    print(df.head())
else:
    print("No stats table found!")



No stats table found!


In [6]:
# assigning all the a tags in the html pages to variable of a links 
links = standings_table.find_all('a')


AttributeError: 'NoneType' object has no attribute 'find_all'

In [ ]:
links = [l.get("href") for l in links]
links = [l for l in links if '/squad/' in l]

links

In [ ]:
team_urls = [f"https://fbref/com{l}" for l in links]

In [ ]:
team_url = team_urls[0]


In [ ]:
data = requests.get(team_url)

data.text

In [ ]:

matches = pd.reaf_html(data.text, match="Scores & Fixtures")
matches[0]

In [ ]:
soup = BeautifulSoup(data.text)

links = soup.find_all('a')

links = [l.get("href") for l in links]


In [ ]:
links  = [l for l in links if l and 'all_comps/shooting/' in l]

In [ ]:
dt = requests.get(f'https://fbref.com{links[0]}')

In [ ]:
data.text

In [ ]:
shooting = pd.read_html(data.text, match="Shooting")[0]


In [ ]:
shooting.head()

In [ ]:
shooting.columns = shooting.columns.droplevel(0)
shooting.head()


In [ ]:
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")


In [ ]:
shooting.shape()

In [7]:
years = list(range(2020, 2024), -1)
years


TypeError: list expected at most 1 argument, got 2

In [8]:
all_matches = []


In [ ]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"